## **Dataclasses**

A dataclass is a decorator in Python (@dataclass) that makes it easier to write classes that are mostly used to store data.


Instead of writing lots of boilerplate code (__ init __ , __ repr __ , __ eq __), Python auto-generates them.

In [ ]:
from dataclasses import dataclass

@dataclass
class Agent:
    name: str
    model: str
    instructions: str

bot = Agent(name="MathBot", model="gpt-4o", instructions="Solve math")
print(bot)   # Agent(name='MathBot', model='gpt-4o', instructions='Solve math')


Agent(name='MathBot', model='gpt-4o', instructions='Solve math')


**Use case:** Perfect for configs, models, structured objects.
In Agents SDK, Agent objects and Tool definitions are dataclasses because they’re just structured data with fields.

In [ ]:
from dataclasses import dataclass, field
# from typing import Optional
@dataclass

class Person:
  name: str
  age: int
  # email: Optional[str] = None
  email: str | None = None

  # using feild() with default_factory for mutable default values
  # tags: List[] # Bad practice
  tags: list[str] = field(default_factory=list)

  def is_adult(self) -> bool:
    """Example method that uses the dataclass attributes."""
    return self.age >= 18

# Usage example
def demo_good_usage():
  # Creatiing instances
  person1 = Person(name= "Alice", age=22, email="alice@example.com")
  person2 = Person(name= "Bob", age=25)
  person3 = Person(name="Charles", age=19, tags=["student", "part-time"])


  # Adding to a mutable filed
  person1.tags.append("developer")

  print(f"Person 1: {person1}")
  print(f"Person 2: {person2}")
  print(f"Person 3: {person3}")


  # Using the instance method
  print(f"Is {person1.name} an adult? {person1.is_adult()}")
  print(f"Is {person3.name} an adult? {person3.is_adult()}")


if __name__ == "__main__":
  print("=== Good Dataclass Example ===")
  demo_good_usage()

=== Good Dataclass Example ===
Person 1: Person(name='Alice', age=22, email='alice@example.com', tags=['developer'])
Person 2: Person(name='Bob', age=25, email=None, tags=[])
Person 3: Person(name='Charles', age=19, email=None, tags=['student', 'part-time'])
Is Alice an adult? True
Is Charles an adult? True


In [ ]:
# Bad Example: Class without dataclass
class PersonBad:
  def __init__(self, name, age, email=None, tags=None):
    self.name = name
    self.age = age
    self.email = email
    # Common mistake: mutable default
    self.tags = tags if tags is not None else[]


  # Have to manually define string represntation
  def __repr__(self):
    return f"PersonBad(name={self.name}, age={self.age}, email={self.email}, tags={self.tags})"

  # Have to manually define equality
  def __eq__(self, other):
    if not isinstance(other, PersonBad):
      return False
    return(self.name == other.name and
           self.age == other.age and
           self.email == other.email and
           self.tags == other.tags)

def demo_bad_usage():
  # More verbose and error-prone without dataclass
  person1 = PersonBad("Alice", 30, "alice@example.come")
  person2 = PersonBad("Bob", 25)

  print(f"Person 1: {person1}")
  print(f"Person 2: {person2}")


if __name__ == "__main__":
  print("=== Bad Dataclass Example ===")
  demo_bad_usage()

=== Bad Dataclass Example ===
Person 1: PersonBad(name=Alice, age=30, email=alice@example.come, tags=[])
Person 2: PersonBad(name=Bob, age=25, email=None, tags=[])


## **Callable**

Callable (from typing) describes something you can call like a function.
This could be:
* a function
* a method
* a class with __call__

When you use it inside a @dataclass, you’re saying: “This field should hold a function with a specific signature.”

**Use case:** In the Agents SDK, tools are often stored as Callables, the agent can call them when needed.

In [ ]:
from dataclasses import dataclass
from typing import Callable

@dataclass
class Calculator:
  operation: Callable[[int, int], int] #function taking (int, int) -> int

def add(x:int, y:int) -> int:
  return x + y

calc = Calculator(operation=add)
print(calc.operation(5,3))

8


``` Callable [[int, int], int] ``` means → function takes two ints, returns int.

add matches this signature, so it works.

## **Generics**

Generics let you write classes or functions that work with many types, while still being type safe.
Instead of fixing a type (like int or str), you use a placeholder type.

They come from typing.Generic and TypeVar.



**Use case:** Generics are used in SDKs and libraries when you want to make flexible but type-safe abstractions.

**Simple Example Without Generics**

In [ ]:
class Box:
    def __init__(self, item):
        self.item = item

int_box = Box(5)
str_box = Box("hello")

print(int_box.item + 2)       # works
print(str_box.item + "!!!")   # works
print(str_box.item + 5)       # runtime error, no warning



**Generics to the Rescue**

In [ ]:
from typing import TypeVar

T = TypeVar("T") # T represents a generic type

* T is a placeholder that can be **replaced with any type** when the function is called.
* The **actual type is inferred at runtime.**

In [ ]:
from typing import TypeVar

# Type variable for generic typing
T = TypeVar('T')

def first_item(items: list[T]) -> T:
  return items[0]

print(first_item([1, 2, 3])) # int
print(first_item["a", "b"]) # str

## **Dataclass VS Pydantic VS Mypy**

## **1. Dataclass**

### **What it is:**
Built-in to Python (from dataclasses import dataclass).
Lets you define plain data containers with less boilerplate.

### **What it gives you:**
* Auto-generates ```__init__, __repr__, __eq__```
* Works well with type hints (but doesn’t enforce them)
* Very lightweight


### **Limitations:**

* No runtime type checking
* If you pass wrong types, it won’t complain

✅ Great for structuring data
❌ Not safe if you need validation

In [ ]:
from dataclasses import dataclass

@dataclass
class User:
    id: int
    name: str

u = User(id="123", name=42)   # ❌ types wrong
print(u)  # Still works! No error until later


## **2. Pydantic**

### **What it is:**
External library (pip install pydantic)
Provides data models with runtime validation and parsing.

### **What it gives you:**

* Validates input at runtime
* Automatically converts types when possible
* Good error messages
* Commonly used in FastAPI, LLM apps, etc.

In [ ]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str

u = User(id="123", name=42)
print(u)   # User id=123 name='42'


**Notice:**
* "123" (string) was converted to 123 (int)
* 42 (int) was converted to "42" (str)

✅ Great for APIs, config, external data
❌ Heavier than dataclass, slower for huge datasets

## **3. Mypy**

### **What it is:**
A static type checker (installed with pip install mypy).
Works at development time, not at runtime.

### **What it gives you:**
* Checks that your code follows type hints
* Detects mismatches before you run the program
* Helps maintain large codebases

### **Limitations:**
* No runtime checks
* Just a linter — if you ignore warnings, your code still runs

✅ Great for developer productivity
❌ Doesn’t protect you at runtime

In [ ]:
def add(x: int, y: int) -> int:
    return x + y

print(add("5", "7"))   # ❌ Runtime: "57" (string concat)

| Feature            | Dataclass ✅            | Pydantic ✅                | Mypy ✅              |
| ------------------ | ---------------------- | ------------------------- | ------------------- |
| Auto `__init__`    | ✅                      | ✅                         | ❌                   |
| Runtime validation | ❌                      | ✅                         | ❌                   |
| Type hints support | ✅ (not enforced)       | ✅ (enforced + casting)    | ✅ (static analysis) |
| Third-party needed | ❌ Built-in             | ✅ Yes                     | ✅ Yes               |
| Best for           | Simple structured data | APIs, configs, user input | Catching bugs early |
